## seeded watershed segmentation
This program uses seeded watershed for grain segmentation.

It takes two input images : a background image to segment and a label image.\
The label image needs to have a pure black background for the program to work.

Explanation of the algorithm : https://youtu.be/LT8L3vSLQ2Q?t=2124

## More information

- argparse : to manage command line arguments.
- os : for manipulating file paths.
- tifffile : for reading and writing TIFF files.
- numpy (alias np) : for manipulating numerical vectors.
- timeit for timer() : to measure execution time.
- hg : an image processing library (a hierarchical processing library).
- skimage : an image processing library (a lot of sub-library to import).

#### French

- argparse : pour gérer les arguments de la ligne de commande.
- os : pour manipuler les chemins de fichiers.
- tifffile : pour lire et écrire des fichiers TIFF.
- numpy (alias np) : pour manipuler les tableaux numériques.
- timeit pour le timer() : pour mesurer le temps d'exécution.
- hg : une bibliothèque de traitement d'images (une bibliothèque de traitement par hiérarchies).
- skimage : une librairie de traitement d'images (énormément de sous-bibliothèques à importer).

# Code

In [ ]:
__author__ = "Lysandre Macke"
# Adapted by Jonathan Palisse
__contact__ = "lmacke@unistra.com"

### Imports

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import higra as hg # ignore the generated warnings

import argparse
import numpy as np
import skimage.morphology as morph
from skimage.measure import label, regionprops
import tifffile # tiff file manipulation
import os

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

from scipy.ndimage import distance_transform_edt

### Command-line program management

Command example : \
python3 segment.py imageToSegment seededImage 6 \
python3 segment.py imageToSegment seededImage 26 \
python3 segment --help

In [ ]:
parser = argparse.ArgumentParser(
                    prog = "segment",
                    description = "test program.")

parser.add_argument("image", help="The path to the tiff file to segment.")
parser.add_argument("seed", help="The path of the tiff file to use as segmentation seed.")
parser.add_argument("adjacency", help="Adjacency of the mask (6 or 26)")

args = parser.parse_args()
image_filepath = args.image
seed_filepath  = args.seed
adjacency = int(args.adjacency)

#### TIFF image loading and resolution check

Seed and image must have the same resolution.

In [ ]:
with tifffile.TiffFile(image_filepath) as tif:
    image = np.array([page.asarray() for page in tif.pages])

with tifffile.TiffFile(seed_filepath) as tif:
    seed = np.array([page.asarray() for page in tif.pages])


if(image.shape != seed.shape):
    print("Error : seed resolution does not match with image !", file=sys.stderr)
    exit()

### Binarization of the image with maxTree

Convert to 8 bits

In [ ]:
image = (image/256).astype("uint8")
seed = (seed/256).astype("uint8")
print("loaded image has shape :", image.shape)

Construction of the graph

In [ ]:
res = np.zeros_like(image)

if (adjacency == 26):
    # 26 adjacency implicit graph
    mask = [[[1, 1, 1], [1, 1, 1], [1, 1, 1]],
            [[1, 1, 1], [1, 0, 1], [1, 1, 1]],
            [[1, 1, 1], [1, 1, 1], [1, 1, 1]]]
else:
    # 6 adjacency implicit graph
    mask = [[[0, 0, 0], [0, 1, 0], [0, 0, 0]],
            [[0, 1, 0], [1, 0, 1], [0, 1, 0]],
            [[0, 0, 0], [0, 1, 0], [0, 0, 0]]]

neighbours = hg.mask_2_neighbours(mask)
graph = hg.get_nd_regular_implicit_graph(image.shape, neighbours)

Construction of the maxTree graph

In [ ]:
tree, altitudes = hg.component_tree_max_tree(graph, image)

# compute attributes
print("Starting max-tree filtering...")
height = hg.attribute_height(tree, altitudes)
area = hg.attribute_area(tree)


# remove unwanted nodes
unwanted_nodes = np.logical_or(height > 100, area < 100)

tree, node = hg.simplify_tree(tree, unwanted_nodes)
new_altitudes = altitudes[node]

Create the binary image

In [ ]:
binary_image = hg.reconstruct_leaf_data(tree, new_altitudes)

binary_image = (binary_image - np.min(image) > 0).astype("uint8")*255  # binarize image

### Preprocessing on the binary image

We get the gradient

In [ ]:
binary_image = morph.area_closing(binary_image)

tifffile.imwrite("tmp_binary.tif", binary_image) # this is our basic maxtree segmentation
print("Created binary image.")

gradient = binary_image
gradient = distance_transform_edt(binary_image) # distance_transform_edt compute the distance-map (each pixel take the value of the distance between it and the nearest black pixel)
gradient = 255 - gradient
tifffile.imwrite("tmp_grad.tif", gradient)

### Compute watershed

Rearrange seeds

In [ ]:
tmp = label(seed > 0, background=0, connectivity=1)     # Pixels of value 0 are considered as background.
                                                        # Connectivity=1 implies that the pixels immediatly adjacent are considered
regions = regionprops(tmp)          # We get information about the regions in the parameter (ex : centroid)

Create centroids image and label it.\
labeled_seeds will contain the value of the centroids of each region

In [ ]:
labeled_seeds = np.zeros_like(seed)

i = 0
for region in regions:
    i += 1
    centroid = np.asarray(region.centroid)
    x, y, z = int(centroid[0]), int(centroid[1]), int(centroid[2])
    labeled_seeds[x, y, z] = tmp[x, y, z]

Use of watershed

In [ ]:
explicit_graph = graph.as_explicit_graph()      # Turn the graph into an explicit graph to apply watershed

edge_weights = hg.weight_graph(explicit_graph, gradient, hg.WeightFunction.mean)        # Compute the weight of the edges of the graph with the gradient.
                                                                                        # higher is the gradient, higher is the weight of the edge.
                                                                                        # The values of the edge will influence the propagation of the watershed
labels = hg.labelisation_seeded_watershed(explicit_graph, edge_weights, labeled_seeds)

res = np.where(binary_image, labels, 0)     # We only want the usefull regions

Creation of the result image

In [ ]:
tifffile.imwrite("result.tif", res)             # Creation of the result image
os.system("python3 colormap.py result.tif")     # Use the colormap.py file to color each region and the background in black

#os.system("gmic colored.tif a z")

exit(0)